In [1]:
import pandas as pd
import numpy as np


In [2]:
train = pd.read_csv('../data/training_set.csv')
train_meta = pd.read_csv('../data/training_set_metadata.csv')

In [17]:
grp = train.groupby(['object_id', 'passband'])
grp.groups

{(615,
  0): Int64Index([ 54,  55,  56,  57,  58,  74, 105, 106, 107, 108, 109, 110, 111,
             122, 128, 129, 130, 131, 132, 133, 154, 155, 176, 177, 178, 179,
             180, 181, 182, 208, 209, 210, 226, 227, 228, 229, 230, 241, 242,
             243, 244, 250, 251, 277, 278, 279, 280, 281, 282, 283, 304, 305,
             306, 307, 308, 309, 310, 336, 337, 338, 339, 340, 341],
            dtype='int64'),
 (615,
  1): Int64Index([  1,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  60,  65,
              70,  76,  81,  86,  91,  96, 101, 113, 118, 124, 135, 140, 145,
             150, 157, 162, 167, 172, 184, 189, 194, 199, 204, 212, 217, 222,
             232, 237, 246, 253, 258, 263, 268, 273, 285, 290, 295, 300, 312,
             317, 322, 327, 332, 343, 348],
            dtype='int64'),
 (615,
  2): Int64Index([  0,   4,   9,  14,  19,  24,  29,  34,  39,  44,  49,  59,  64,
              69,  75,  80,  85,  90,  95, 100, 112, 117, 123, 134, 139, 144,
             14